In [1]:
import pandas as pd
import ast
import json
import re
import numpy as np

In [2]:
%cd

/Users/kyuheekim


/opt/anaconda3/envs/ada/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/opt/anaconda3/envs/ada/lib/python3.11/site-packages/IPython/core/magics/osm.py:428: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Check DataFrame

In [3]:
df = pd.read_csv('./STORY/data/raw_movie_metadata.csv')
df.head(3)

,Unnamed: 0,id,title,release_date,revenue,runtime,adult,budget,overview,popularity,poster_path,genres,keywords,year,actors,emotions,director
0,0,565770,Blue Beetle,2023-08-16,124818235,128,False,120000000,Recent college grad Jaime Reyes returns home f...,2994.357,/mXLOHHc1Zeuwsl4xYKjKh2280oL.jpg,"Action, Science Fiction, Adventure","armor, superhero, family relationships, family...",2023,"['Xolo Mariduena', 'Bruna Marquezine', 'Susan ...","['excitement', 'pride', 'inspiration']",Ángel Manuel Soto
1,1,980489,Gran Turismo,2023-08-09,114800000,135,False,60000000,The ultimate wish-fulfillment tale of a teenag...,2680.593,/51tqzRtKMMZEYUpSYkrUE7v9ehm.jpg,"Action, Drama, Adventure","based on true story, racing, based on video ga...",2023,"['Archie Madekwe', 'David Harbour', 'Orlando B...","['tension', 'determination', 'hope']",Neill Blomkamp
2,2,754720,A Female Boss with Big Tits and Her Cherry Boy...,2020-01-16,0,120,True,0,Yuzuru is this clumsy permavirgin employee who...,2020.286,/upVlyO6Zc5AecmTNv0XZ1oiryZf.jpg,Drama,"cheating, office, big tits, unfaithful wife",2020,"['Yua Mikami', 'Yuki Yuduru']","['lust', 'taboo', 'thrill']",Kitorune Kawaguchi


### Filter Adult

In [4]:
df = df[~df['adult']]

### Genres

In [5]:
malformed = df[df['genres'].apply(lambda x: bool(re.match(r'[^A-Za-z]', str(x))))]['genres']
set(malformed)

{"['Action', 'Comedy', 'Crime']",
 "['Action', 'Crime']",
 "['Action', 'Drama', 'Thriller', 'Crime']",
 "['Action']",
 "['Adventure', 'Animation', 'Family', 'Fantasy']",
 "['Adventure', 'Drama', 'History']",
 "['Adventure', 'Drama']",
 "['Animation', 'Adventure', 'Comedy', 'Family']",
 "['Animation', 'Fantasy', 'Mystery', 'Horror', 'Action']",
 "['Comedy', 'Crime']",
 "['Comedy', 'Drama']",
 "['Comedy', 'Music']",
 "['Comedy', 'Romance']",
 "['Comedy']",
 "['Crime', 'Drama', 'Romance']",
 "['Documentary']",
 "['Drama', 'Comedy']",
 "['Drama', 'Crime']",
 "['Drama', 'Family']",
 "['Drama', 'Horror']",
 "['Drama', 'Thriller', 'Romance']",
 "['Drama', 'Thriller']",
 "['Drama']",
 "['Family', 'Comedy', 'Adventure', 'Music']",
 "['Fantasy', 'Action', 'Adventure']",
 "['History', 'Drama']",
 "['Horror', 'Comedy']",
 "['Horror', 'Drama', 'Fantasy']",
 "['Horror', 'Fantasy', 'Comedy']",
 "['Horror', 'Thriller']",
 "['Music', 'Romance', 'Drama']",
 "['Romance']",
 "['Thriller', 'Horror']",
 '[]

In [6]:
def normalize_genre_string(val):
    if pd.isna(val) or val.strip() in ("[]", "", "['']", "None", "nan"):
        return np.nan

    # Case 1: Looks like a Python list
    if val.strip().startswith("[") and val.strip().endswith("]"):
        try:
            parsed = ast.literal_eval(val)
            if isinstance(parsed, list):
                return [g.strip() for g in parsed if g.strip()]
        except:
            pass

    # Case 2: Just a comma-separated string
    return [g.strip() for g in val.split(",") if g.strip()]

In [7]:
df['genres'] = df['genres'].apply(normalize_genre_string)

In [8]:
df['genres'] = df['genres'].apply(str)

### Keywords

In [9]:
malformed = df[df['keywords'].apply(lambda x: bool(re.match(r'[^A-Za-z]', str(x))))]['keywords']
set(malformed)

{'1970s, alien, train crash, firecracker, super 8mm footage, duringcreditsstinger',
 '1970s, anti hero, villain, punk rock, fashion designer, fashion, origin story, live action remake',
 '1970s, drug addiction, punk rock, based on true story, sex pistols, co-workers relationship, punk band, sid vicious, music tour, music manager, chelsea hotel, musical acts, toxic relationship',
 '1970s, fbi, based on true story, ohio, heist, richard nixon, theft, investigator, watergate, bank heist',
 '1970s, female friendship, coming of age, gynecologist, photocopier, truth or dare, woman director',
 '1970s, journalism, sexism, ladder, panda, tv show in film, mustache, screwball comedy, battle of the sexes, misogynist, teleprompter, gang warfare, aftercreditsstinger, duringcreditsstinger, news spoof',
 '1970s, louisiana, kansas, usa, guitar player, awol, target practice, exhaustion, military discharge, stockade',
 '1970s, mutant, time travel, based on comic, superhuman, storm, political intrigue\xa0,

### Emotions

In [10]:
malformed = df[df['emotions'].apply(lambda x: bool(re.match(r'[^A-Za-z]', ', '.join(eval(x)))))]['emotions']
set(malformed)

set()

### Director

In [11]:
malformed = df[df['director'].apply(lambda x: bool(re.match(r'[^A-Za-z]', str(x))))]['director']
set(malformed)

{'Álex de la Iglesia',
 'Ángel Gómez Hernández',
 'Ángel Manuel Soto',
 'Éric Rohmer'}

# Cleanse

In [12]:
def clean_malformed(val):
    if pd.isna(val):
        return np.nan

    # Case 1: Actual empty list
    if isinstance(val, list) and len(val) == 0:
        return np.nan

    # Case 2: String representations of malformed values
    if isinstance(val, str):
        val = val.strip()
        if val in ("", "[]", "['']", "nan", "None"):
            return np.nan

    return val

In [13]:
df = df.applymap(clean_malformed)
df

/var/folders/4t/fm4_3xrd5d7d78c65jqr52c00000gn/T/ipykernel_830/4047936869.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(clean_malformed)


,Unnamed: 0,id,title,release_date,revenue,runtime,adult,budget,overview,popularity,poster_path,genres,keywords,year,actors,emotions,director
0,0,565770,Blue Beetle,2023-08-16,124818235,128,False,120000000,Recent college grad Jaime Reyes returns home f...,2994.357,/mXLOHHc1Zeuwsl4xYKjKh2280oL.jpg,"['Action', 'Science Fiction', 'Adventure']","armor, superhero, family relationships, family...",2023,"['Xolo Mariduena', 'Bruna Marquezine', 'Susan ...","['excitement', 'pride', 'inspiration']",Ángel Manuel Soto
1,1,980489,Gran Turismo,2023-08-09,114800000,135,False,60000000,The ultimate wish-fulfillment tale of a teenag...,2680.593,/51tqzRtKMMZEYUpSYkrUE7v9ehm.jpg,"['Action', 'Drama', 'Adventure']","based on true story, racing, based on video ga...",2023,"['Archie Madekwe', 'David Harbour', 'Orlando B...","['tension', 'determination', 'hope']",Neill Blomkamp
3,3,968051,The Nun II,2023-09-06,231200000,110,False,38500000,"In 1956 France, a priest is violently murdered...",1692.778,/c9kVD7W8CT5xe4O3hQ7bFWwk68U.jpg,"['Horror', 'Mystery', 'Thriller']","france, bullying, sequel, religion, demon, got...",2023,"['Taissa Farmiga', 'Jonas Bloquet', 'Storm Rei...","['fear', 'suspense', 'dread']",Michael Chaves
4,4,615656,Meg 2: The Trench,2023-08-02,384056482,116,False,129000000,An exploratory dive into the deepest depths of...,1567.273,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,"['Action', 'Science Fiction', 'Horror']","based on novel or book, sequel, shark, kaiju, ...",2023,"['Jason Statham', 'Wu Jing', 'Shuya Sophia Cai...","['fear', 'adrenaline', 'awe']",Ben Wheatley
5,5,762430,Retribution,2023-08-23,12905464,91,False,20000000,When a mysterious caller puts a bomb under his...,1547.220,/oUmmY7QWWn7OhKlcPOnirHJpP1F.jpg,"['Action', 'Mystery', 'Thriller', 'Crime']","assassination, bomb, remake, divorce, duringcr...",2023,"['Liam Neeson', 'Lilly Aspell', 'Jack Champion...","['anger', 'revenge', 'justice']",Nimród Antal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,17414,Dunston Checks In,1996-01-12,0,88,False,16000000,Hotel manager Robert Grant is forced by his bo...,10.671,/mvbJSNMRzM7epjeJMT3vFxXydQJ.jpg,"['Family', 'Comedy', 'Adventure']","boss, fistfight, bully, orangutan, single fath...",1996,"['Sam the Orangutan', 'Jason Alexander', 'Faye...","['comedy', 'chaos', 'friendship']",Ken Kwapis
7996,7996,47504,Little Odessa,1994-10-28,0,98,False,0,"Long separated from his Russian family, hitman...",10.670,/80zDn3oanRMWDSDX1cPKZVIi8OO.jpg,"['Crime', 'Drama']","bratva (russian mafia), brother, dysfunctional...",1994,"['Tim Roth', 'Edward Furlong', 'Moira Kelly', ...","['coldness', 'family', 'crime']",James Gray
7997,7997,1436,The Warrior,2001-09-07,0,155,False,8000000,"In 1375, China was in chaos between Yuan Dynas...",10.668,/i0lpQfqQHN7vEYzSHOk6B65dQbb.jpg,"['History', 'Drama', 'Action', 'War']","ambassador, china, hostility, gobi desert, mon...",2001,"['Jung Woo-sung', 'Ahn Sung-ki', 'Ju Jin-mo', ...","['destiny', 'violence', 'redemption']",Kim Sung-soo
7998,7998,492675,Nude Bowling Party,1995-01-15,0,55,False,0,"Two teams, one consisting of two models and on...",10.667,/gOcVtmfQhQXNR8TY3MRwgRxODHC.jpg,['Comedy'],sports,1995,"['Teresa Politi', 'Taryn Carter', 'Jacqueline ...","['absurdity', 'humor', 'nudity']",NaN


In [14]:
df.to_csv('./STORY/data/movie_metadata.csv')